# Multi-Document & Multi-Agent RAG

## Import

In [3]:
from pathlib import Path
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager
import re
from glob import glob
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

## Preproceesing Tools

This function makes sure the files namming are compatabile to be used as vector index

In [6]:
# Helper function to sanitize names
def sanitize_name(name: str) -> str:
    # Replace spaces with underscores
    name = name.replace(" ", "_")
    # Remove any character that is not alphanumeric, underscore, or dash
    return re.sub(r"[^a-zA-Z0-9_-]", "", name)

In [7]:
def merge_consecutive_headers(lines):
    """
    Processes a list of lines from a Markdown file and merges consecutive header lines.
    A header is defined as a line starting with 1-6 '#' characters followed by a space.
    When two or more header lines are consecutive, they are merged into a single header.
    The first header's level is preserved and the content from the subsequent headers is appended.
    """
    merged_lines = []
    i = 0
    while i < len(lines):
        line = lines[i].rstrip("\n")
        if re.match(r'^#{1,6}\s', line):
            header_line = line
            j = i + 1
            while j < len(lines) and re.match(r'^#{1,6}\s', lines[j]):
                match = re.match(r'^(#{1,6})\s+(.*)', lines[j])
                if match:
                    header_text = match.group(2).strip()
                    header_line = header_line.rstrip() + " " + header_text
                j += 1
            merged_lines.append(header_line + "\n")
            i = j  # Skip all merged header lines
        else:
            merged_lines.append(line + "\n")
            i += 1
    return merged_lines

def clean_markdown_files(input_directory, output_directory=None):
    """
    Iterates through all Markdown (.md) files in the input_directory,
    merges consecutive header lines in each file, and writes the cleaned content
    to the output_directory. If output_directory is not provided, a subfolder
    called 'cleaned' will be created inside the input_directory.
    """
    if output_directory is None:
        output_directory = os.path.join(input_directory, "cleaned")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    md_files = glob(os.path.join(input_directory, "*.md"))
    for md_file in md_files:
        with open(md_file, "r", encoding="utf-8") as f:
            lines = f.readlines()
        cleaned_lines = merge_consecutive_headers(lines)
        output_file = os.path.join(output_directory, os.path.basename(md_file))
        with open(output_file, "w", encoding="utf-8") as f:
            f.writelines(cleaned_lines)
        print(f"Cleaned file written to: {output_file}")


This function merges consecutiv header to fix chunking issues. Call the function and set the correct path for the files. 
After running, a file named 'cleaned' will be created in the same directory of the fiels, use the files in the cleaned folder as the path for following steps

In [ ]:
# input_dir = ".\التقارير السنوية" # e.g., "./markdown_files"

# # Clean the Markdown files by merging consecutive headers.
# clean_markdown_files(input_dir)

## Setting Up LLM & Embedding Models

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
Settings.llm = OpenAI(temperature=0, model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Make sure the path here is the path of cleaned files from previous step

In [4]:
report_types = {
    "annual_reports": "Data/التقارير السنوية",        # Folder containing 8 markdown files
    "balance_of_banks": "Data/ميزان المدفوعات",        # Folder containing 10 markdown files
}

## Creating Lowe-Level Agents With Query Engine Tools 

In [8]:
# Prepare dictionaries to hold low-level agents and their top-level tools
low_level_agents = {}
low_level_tools = {}

# We'll use a SentenceSplitter to break up the markdown text into nodes
from llama_index.core.node_parser import MarkdownNodeParser
node_parser = MarkdownNodeParser()

# For each report type, build file-level tools and then group them into a low-level agent.
for report_type, folder_path in report_types.items():
    # Sanitize the report type name for tool naming
    safe_report_type = sanitize_name(report_type)
    
    # Get a list of markdown files in the directory
    file_paths = [str(path) for path in Path(folder_path).glob("*.md")]
    file_tools = []  # This will hold a tool per file in the report type

    for file_path in file_paths:
        # Load the markdown document from the file
        docs = SimpleDirectoryReader(input_files=[file_path]).load_data()
        # Split the document into nodes for finer granularity
        nodes = node_parser.get_nodes_from_documents(docs)
        
        # Create or load a vector index for this file
        # We persist the index in a subdirectory named after the file (without extension)
        file_stem = Path(file_path).stem
        safe_file_stem = sanitize_name(file_stem)
        persist_dir = f"./data/{safe_report_type}/{safe_file_stem}"
        if not os.path.exists(persist_dir):
            vector_index = VectorStoreIndex(nodes)
            vector_index.storage_context.persist(persist_dir=persist_dir)
        else:
            from llama_index.core import load_index_from_storage, StorageContext
            vector_index = load_index_from_storage(
                StorageContext.from_defaults(persist_dir=persist_dir)
            )
        
        # Create the vector query engine for this file
        vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
        
        # Wrap the query engine in a tool, labeling it with the sanitized file name
        file_tool = QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"tool_{safe_report_type}_{safe_file_stem}",
                description=f"Tool for the report file '{file_stem}' in {report_type.replace('_', ' ')}."
            ),
        )
        file_tools.append(file_tool)
    
    # Create a low-level agent for this report type that aggregates all file-level tools
    from llama_index.agent.openai import OpenAIAgent
    function_llm = OpenAI(model="gpt-4o")
    low_level_agent = OpenAIAgent.from_tools(
        file_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=(
            f"You are a specialized agent designed to answer queries about {report_type.replace('_', ' ')}. "
            "You must ALWAYS use one of the provided tools and never rely on prior knowledge."
        ),
    )
    low_level_agents[report_type] = low_level_agent
    
    # Create a top-level tool for this low-level agent with a sanitized name
    report_tool = QueryEngineTool(
        query_engine=low_level_agent,
        metadata=ToolMetadata(
            name=f"agent_tool_{safe_report_type}",
            description=f"Tool for answering queries about {report_type.replace('_', ' ')} financial reports."
        ),
    )
    low_level_tools[report_type] = report_tool


In [9]:
all_tools = list(low_level_tools.values())

## Setting Up Top-Level Agent: Retriever

This step creates an index for the tools to be treated as text so that they can be retrieved

In [10]:
# Create an ObjectIndex over the low-level agent tools
from llama_index.core.objects import ObjectIndex
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [11]:
# Build the top-level agent that retrieves the right low-level agent based on the query
from llama_index.agent.openai import OpenAIAgent
top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=(
        "You are a top-level agent designed to answer queries about financial data from multiple report types. "
        "Based on the query, select the appropriate tool to use (e.g., annual reports or balance of banks)."
        "Always use the selected tool and never rely on prior knowledge."
        "Queries should be phrased in the same language as the prompt."
        "Always respond in arabic"
        "If the response is from the tool is not optimal, change the tool untill you find satisfactory response"
    ),
    verbose=True,
)

### Examples for one top-level OpenAI Agent

In [29]:
response = top_agent.query("ماهي نسبة البطالة في التقارير السنوية لعام 2010؟")

Added user message to memory: ماهي نسبة البطالة في التقارير السنوية لعام 2010؟
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input":"نسبة البطالة في التقارير السنوية لعام 2010"}
Added user message to memory: نسبة البطالة في التقارير السنوية لعام 2010
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2010 with args: {
  "input": "unemployment rate"
}
Got output: The unemployment rates in industrialized countries saw a noticeable increase in 2010 compared to 2009. The rates were 8.3% in 2010, up from 8.0% in 2009. In the Eurozone countries, the rate rose from 9.4% to 10.0%. The United States saw an increase from 9.3% in 2009 to 9.6% in 2010, while the United Kingdom's rate went up from 7.5% to 7.8%. Japan's unemployment rate remained unchanged at 5.1%.

Got output: معدلات البطالة في البلدان المصنعة شهدت زيادة ملحوظة في عام 2010 مقارنة بعام 2009. كانت النسبة 8.3% في عام 2010، مقابل 8.0% في عام 2009. في دول منطقة اليورو، ارتفعت

In [40]:
print(response.source_nodes[0])

Node ID: 55c0aa8d-4aaa-412e-ad21-fbbcd91fd517
Text: ## البطالة •    شهدت معدلات البطالة في الدول الصناعية عام 2010
زيادة ملحوظة حيث بلغت 8.3% مقابل 8.0% عام 2009، وارتفعت في دول منطقة
اليورو من 9.4% إلى 10.0%. كما ارتفع معدل البطالة في الولايات المتحدة
الأمريكية من 9.3% في عام 2009 إلى 9.6% في عام 2010، وفي المملكة
المتحدة من 7.5% إلى 7.8%. أما معدل البطالة في اليابان فقد ظل على ما هو
عليه عند 5.1%.
Score:  0.404



In [63]:
response = top_agent.query("قارن بين معدلات التضخم في التقارير السنوية لعام 2013 وعام 2014")

Added user message to memory: قارن بين معدلات التضخم في التقارير السنوية لعام 2013 وعام 2014
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input": "معدل التضخم لعام 2013"}
Added user message to memory: معدل التضخم لعام 2013
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2013 with args: {"input":"معدل التضخم لعام 2013"}
Got output: معدل التضخم لعام 2013 هو 6.2%.

Got output: معدل التضخم لعام 2013 هو 6.2%.

=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input": "معدل التضخم لعام 2014"}
Added user message to memory: معدل التضخم لعام 2014
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2014 with args: {"input":"ما هو معدل التضخم لعام 2014؟"}
Got output: معدل التضخم لعام 2014 بلغ 2.4% وفقاً لمؤشر الرقم القياسي العام لأسعار المستهلك.

Got output: معدل التضخم لعام 2014 بلغ 2.4% وفقاً لمؤشر الرقم القياسي العام لأسعار المستهلك.



In [59]:
print(response.source_nodes[0])

Node ID: 53401fa8-6b80-47d4-acd7-eaedb8525a95
Text: ## معدل التضخم :    استناداً إلى بيانات مصلحة الإحصاء والتعداد،
فإن معدل التضخم وفقاً لمؤشر الرقم القياسي العام لأسعار المستهلك بلغ
2.4% في نهاية عام 2014، حيث ارتفع الرقم القياسي لأسعار المستهلك من
163.7 عام 2013 إلى 167.7 عام 2014. و تركز هذا الارتفاع في أسعار مجموعة
المواد الغذائية والمشروبات والتبغ بنسبة 4.5%، وارتفع الرقم القياسي
لمجموعة ال...
Score:  0.636



In [18]:
response = top_agent.query("ماهي النسبة المئوية للناتج المحلي الإجمالي في التقارير السنوية لعام 2013؟")

Added user message to memory: ماهي النسبة المئوية للناتج المحلي الإجمالي في التقارير السنوية لعام 2013؟
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input":"الناتج المحلي الإجمالي لعام 2013"}
Added user message to memory: الناتج المحلي الإجمالي لعام 2013
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2013 with args: {"input":"الناتج المحلي الإجمالي لعام 2013"}
Got output: الناتج المحلي الإجمالي الاسمي لعام 2013 سجل ارتفاعًا بنسبة 20.8%، حيث بلغت قيمة الناتج المحلي النفطي 4284 مليار دينار، بينما بلغ الناتج المحلي غير النفطي حوالي 348 مليار دينار.

Got output: الناتج المحلي الإجمالي الاسمي لعام 2013 سجل ارتفاعًا بنسبة 20.8%. بلغت قيمة الناتج المحلي النفطي 4284 مليار دينار، بينما بلغ الناتج المحلي غير النفطي حوالي 348 مليار دينار.



## Creating Agent Runner & Agent Worker with loop reasoning

In [ ]:
obj_retreiver = obj_index.as_retriever(similarity_top_k=3)

In [25]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
llm=OpenAI(model="gpt-4o",
    system_prompt="You are a top-level agent designed to answer queries about financial data from multiple report types."
        "Based on the query, select the appropriate tool to use (e.g., annual reports or balance of banks)."
        "Always use the selected tool and never rely on prior knowledge."
           )
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retreiver,
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [20]:
response = agent.query("قارن بين معدلات التضخم في التقارير السنوية لعام 2013 وعام 2014")

Added user message to memory: قارن بين معدلات التضخم في التقارير السنوية لعام 2013 وعام 2014
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input": "\u0645\u0639\u062f\u0644 \u0627\u0644\u062a\u0636\u062e\u0645 \u0641\u064a \u0627\u0644\u062a\u0642\u0631\u064a\u0631 \u0627\u0644\u0633\u0646\u0648\u064a \u0644\u0639\u0627\u0645 2013"}
Added user message to memory: معدل التضخم في التقرير السنوي لعام 2013
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2013 with args: {"input":"معدل التضخم"}
Got output: في عام 2013، ارتفع معدل التضخم ليصل إلى 6.1% مقارنة بـ 2.6% في عام 2012. هذا الارتفاع يعزى إلى زيادة الإنفاق العام، وخاصة الإنفاق الجاري.

=== Function Output ===
في عام 2013، ارتفع معدل التضخم ليصل إلى 6.1% مقارنة بـ 2.6% في عام 2012. هذا الارتفاع يعزى إلى زيادة الإنفاق العام، وخاصة الإنفاق الجاري.
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input": "\u0645\u0639\u062f\u0644 \u0627\u0644\

In [27]:
response = agent.query("في عام 2008, ماهي نسبة البطالة في امريكا وماهي قيمة الحساب الجاري في ليبيا؟")

Added user message to memory: في عام 2008, ماهي نسبة البطالة في امريكا وماهي قيمة الحساب الجاري في ليبيا؟
=== Calling Function ===
Calling function: agent_tool_annual_reports with args: {"input": "\u0646\u0633\u0628\u0629 \u0627\u0644\u0628\u0637\u0627\u0644\u0629 \u0641\u064a \u0627\u0645\u0631\u064a\u0643\u0627 \u0639\u0627\u0645 2008"}
Added user message to memory: نسبة البطالة في امريكا عام 2008
=== Calling Function ===
Calling function: tool_annual_reports_annual_report_2008 with args: {"input":"نسبة البطالة في امريكا عام 2008"}
Got output: نسبة البطالة في الولايات المتحدة الأمريكية عام 2008 كانت 5.8%.

=== Function Output ===
نسبة البطالة في الولايات المتحدة الأمريكية عام 2008 كانت 5.8%.
=== Calling Function ===
Calling function: agent_tool_balance_of_banks with args: {"input": "\u0642\u064a\u0645\u0629 \u0627\u0644\u062d\u0633\u0627\u0628 \u0627\u0644\u062c\u0627\u0631\u064a \u0641\u064a \u0644\u064a\u0628\u064a\u0627 \u0639\u0627\u0645 2008"}
Added user message to memory: قيمة 